In [ ]:
###
# Pull Data From FRED and generate more useful visualizations and analysis
# Date: Fall 2022
# Author: Alex Parker
###


# Package Import
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px 
from datetime import datetime
from datetime import date
from datetime import timedelta
import fredapi as fa
import plotly.graph_objects as go
from plotly.subplots import make_subplots

: 

In [2]:
# Connect to FRED API, you can get an API key on the website after making a free account
fred_api_key = '0a41adbcb8d809cddc65754f65d615bf'
fred = fa.Fred(api_key = fred_api_key)


In [3]:
# Function to grab FRED series and convert to a nice dataframe with relevant columns
def get_fred_data(fred_code, metric_name, geo):
    data = fred.get_series(fred_code)
    data = (pd.DataFrame(data)
        .reset_index()
        .rename(columns = {'index':'date',0:metric_name})
        .assign(geo = geo,
                year = lambda x: x.date.dt.year,
                month = lambda x: x.date.dt.month)
        .astype({'geo':'category'})
            )    
    data = data[['date','year','month','geo',metric_name,]]
    return(data)

### Dual Y axis Plotting function
def dual_yaxis_lineplot(x, y1, y2,plot_title, x_title, y1_title, y2_title):
    
    # initiate a dual y axes subplot
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add trace: 1st and 2nd lines
    fig = (fig
            .add_trace(
                go.Scatter(x=x, y=y1, name= y1_title),
                secondary_y=False)
            .add_trace(
                go.Scatter(x=x, y=y2, name= y2_title),
                secondary_y=True)
            )

    # Add figure titles
    fig = (fig.update_layout(title_text= plot_title)
        # Set x-axis title
        .update_xaxes(title_text= x_title)
        # Set y-axes titles
        .update_yaxes(title_text= y1_title, secondary_y= False)
        .update_yaxes(title_text= y2_title, secondary_y= True)
            )
    return(fig)



In [4]:
print(fa.__version__)

0.4.2


In [9]:
gdp = fred.search("GDP")

AttributeError: 'xml.etree.ElementTree.Element' object has no attribute 'getchildren'

In [5]:
### Pull in some macro data
rgdp = get_fred_data(fred_code = 'GDPC1', metric_name = "rgdp", geo = 'US')

# Labor  Force
unemployment = get_fred_data(fred_code = 'UNRATE', metric_name = "unemployment", geo = 'US')
lfpr = get_fred_data(fred_code = 'CIVPART', metric_name = "lfpr", geo = 'US')

# Money Supply
m2 = get_fred_data(fred_code = 'M2SL', metric_name = "money_supply", geo = 'US')
m2v = get_fred_data(fred_code = 'M2V', metric_name = "m2v", geo = 'US')

# Prices
cpi_urb = get_fred_data(fred_code = 'CPIAUCSL', metric_name = 'cpi_urb', geo = 'US')
cpi_urb_less_fe = get_fred_data(fred_code = 'CPILFESL', metric_name = 'cpi_urb_less_fe', geo = 'US')

# Interest Rates
ffr = get_fred_data(fred_code = 'FEDFUNDS', metric_name = 'federal_funds_rate', geo = 'US')
#mortgage = get_fred_data(fred_code = 'MORTGAGE30US', metric_name = 'mortgage_rate', geo = 'US')

# Commodity Prices
#wti_oil_price = get_fred_data(fred_code = 'DCOILWTICO', metric_name = 'wti_oil_price', geo = 'US')
global_energy = get_fred_data(fred_code = 'PNRGINDEXM', metric_name = 'global_energy_price_index', geo = 'GLOBAL')
global_materials = get_fred_data(fred_code = 'PRAWMINDEXM', metric_name = 'global_material_price_index', geo = 'GLOBAL')
global_food = get_fred_data(fred_code = 'PFOODINDEXM', metric_name = 'global_food_price_index', geo = 'GLOBAL')

AttributeError: 'xml.etree.ElementTree.Element' object has no attribute 'getchildren'

In [ ]:
### Real GDP Plots

rgdp = (rgdp
        .assign(rgdp_growth_qq = lambda x: x.rgdp.pct_change(periods = 1*100),
                rgdp_growth_yy = lambda x: x.rgdp.pct_change(periods = 4)*100)
        )

### Real GDP Plot
fig = dual_yaxis_lineplot(x = rgdp.date, 
                        y1 = rgdp.rgdp_growth_yy, 
                        y2 = rgdp.rgdp,
                        plot_title = "Real GDP in the US",
                        x_title = 'Date',
                        y1_title = 'Growth Rate Y/Y (%)',
                        y2_title = 'Real GDP (Billions $)')

fig = (fig.update_layout(legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01))
        .add_hline(y = 0)
        )

fig.show()

In [ ]:
### Labor Market Plots
fig = make_subplots(
        rows=1, cols=2,
        subplot_titles= ("US Unemployment Rate","US Labor Force Participation"))

fig.add_trace(
    go.Scatter(x=unemployment.date, y=unemployment.unemployment, name = 'Unemployment Rate'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=lfpr.date, y=lfpr.lfpr, name = 'Labor force Participation'),
    row=1, col=2
)

fig.update_layout(title_text="Labor Market Essential Metrics")
fig.update_yaxes(title_text="Unemployment Rate (%)", row=1, col=1)
fig.update_yaxes(title_text="Labor Force Participation Rate (%)", row=1, col=2)

fig.update_layout(legend=dict(
            yanchor="bottom",
            y=-0.25,
            xanchor="left",
            x=0.01))

fig.show()

In [ ]:
### Consumer Price Index Plots
fig = make_subplots(
        rows=1, cols=2,
        subplot_titles= ("CPI: Urban Consumers","CPI Less Food & Energy: City Consumers"))

fig.add_trace(
    go.Scatter(x = cpi_urb.date, y = cpi_urb.cpi_urb, name = 'CPI'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=cpi_urb.date, y= cpi_urb_less_fe.cpi_urb_less_fe, name = 'CPI Less Food & Energy'),
    row=1, col=2
)

fig.update_layout(title_text="Consumer Prices Indices")
fig.update_yaxes(title_text="CPI", row=1, col=1)
fig.update_yaxes(title_text="CPI Less Food & Energy", row=1, col=2)

fig.update_layout(legend=dict(
            yanchor="bottom",
            y=-0.25,
            xanchor="left",
            x=0.01))

fig.show()